In [1]:

import socket

def verify_esp32_connection():
    esp_ip = '192.168.31.222'
    esp_port = 80
    timeout = 3  # seconds
    
    try:
        with socket.create_connection((esp_ip, esp_port), timeout=timeout):
            print(f"Success: ESP32 reachable at {esp_ip}:{esp_port}")
            return True
    except Exception as e:
        print(f"Connection failed: {str(e)}")
        print("Check:")
        print("1. ESP32 is powered ON")
        print("2. Correct IP address")
        print("3. Same network for PC/ESP32")
        print("4. No firewall blocking port 80")
        return False

if not verify_esp32_connection():
    exit()
    
    

Success: ESP32 reachable at 192.168.31.222:80


In [1]:
import torch
import cv2
import requests
import numpy as np
from PIL import Image
from ultralytics import YOLO
from io import BytesIO
import time

# Check GPU availability
print(f"CUDA available: {torch.cuda.is_available()}")
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Configuration
ESP_IP = "192.168.31.222"
STREAM_URL = f"http://{ESP_IP}/stream"
MODEL_PATH = "yolov8n.pt"
MAX_RETRIES = 5
TIMEOUT = 15

# Load model with GPU optimization
model = YOLO(MODEL_PATH).to(device)
model.fuse()  # Fuse conv+bn layers for faster inference

# GPU-specific optimizations
if device == 'cuda':
    model.half()  # FP16 precision
    torch.backends.cudnn.benchmark = True

print(f"Model loaded on: {next(model.parameters()).device}")

def process_stream():
    retry_count = 0
    session = requests.Session()
    
    while retry_count < MAX_RETRIES:
        try:
            response = session.get(STREAM_URL, stream=True, timeout=TIMEOUT)
            bytes_data = b''
            
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    bytes_data += chunk
                    a = bytes_data.find(b'\xff\xd8')
                    b = bytes_data.find(b'\xff\xd9')
                    
                    if a != -1 and b != -1:
                        jpg = bytes_data[a:b+2]
                        bytes_data = bytes_data[b+2:]
                        
                        try:
                            # GPU-accelerated processing
                            img = Image.open(BytesIO(jpg))
                            frame = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
                            
                            # GPU inference with optimizations
                            results = model.track(
                                frame,
                                imgsz=640,
                                conf=0.5,
                                device=device,
                                half=(device == 'cuda'),
                                verbose=False,
                                persist=True
                            )
                            
                            # Display results
                            annotated_frame = results[0].plot()
                            cv2.imshow('ESP32-CAM Detection', annotated_frame)
                            if cv2.waitKey(1) == ord('q'):
                                return
                            
                            retry_count = 0
                            
                        except Exception as e:
                            print(f"Frame error: {str(e)}")
                            continue
                            
        except (requests.exceptions.RequestException, KeyboardInterrupt):
            retry_count += 1
            print(f"Connection lost. Retry {retry_count}/{MAX_RETRIES}")
            time.sleep(2)
            
        finally:
            cv2.destroyAllWindows()

    print("Max retries exceeded. Exiting.")

if __name__ == "__main__":
    process_stream()
    

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/home/beast095/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
CUDA available: True



00%|██████████████████████████████████████| 6.25M/6.25M [00:01<00:00, 3.37MB/s]

YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
Model loaded on: cuda:0
requirements: Ultralytics requirement ['lap>=0.5.12'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 2.1 MB/s eta 0:00:002.0 MB/s eta 0:00:01

requirements: AutoUpdate success ✅ 2.7s, installed 1 package: ['lap>=0.5.12']
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect

WARNING ⚠️ not enough matching points
Connection lost. Retry 1/5
Connection lost. Retry 1/5
Connection lost. Retry 1/5
Connection lost. Retry 1/5
Connection lost. Retry 1/5
Connection lost. Retry 2/5


KeyboardInterrupt: 